In [14]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Fix Nan error in std measured for the metrics (use the prdicted scores recorded)

In [18]:
model_results_dir = glob.glob('./logged_performance_per_model/*')

In [26]:
for model_dir in model_results_dir:
    prediction_filepaths = glob.glob(f'{model_dir}/prediction_scores/*.csv')

    for prediction_filepath in prediction_filepaths:
        analysis_unit = 2
        break
    break

In [27]:
prediction_filepath.split('/')[-1].split()

'./logged_performance_per_model/distilbert-base-multilingual-cased/prediction_scores/multilingual_fit_chunked_pred_scores_test_fold_3_po.csv'

In [15]:
pred_scores = glob.glob('./logged_performance_per_model/*/prediction_scores/*.csv')

In [24]:
a = pd.read_csv( './logged_performance_per_model/EleutherAI-gpt-neo-125M/prediction_scores/multilingual_fit_chunked_pred_scores_test_fold_2_ru.csv', index_col='id')
a

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
id,,,,,,,,,,,,,,
241,0.000029,0.340945,0.000147,0.115598,0.000210,0.002918,0.004900,0.017213,0.002128,0.000176,0.060845,0.000742,0.027296,0.001883
243,0.000980,0.102169,0.000108,0.000400,0.000049,0.000352,0.000045,0.001222,0.001895,0.000819,0.003158,0.000147,0.000428,0.468434
2413,0.070560,0.000855,0.000172,0.975025,0.017289,0.036765,0.008565,0.004318,0.001351,0.000425,0.006484,0.085441,0.054763,0.034473
2413,0.000135,0.128284,0.070594,0.040483,0.000840,0.000268,0.000433,0.032315,0.094752,0.000014,0.004483,0.002199,0.000221,0.006018
2415,0.003161,0.045200,0.000372,0.000074,0.005121,0.000032,0.000032,0.000117,0.000129,0.000270,0.002908,0.000059,0.000053,0.997557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24178,0.000434,0.008445,0.003384,0.000294,0.010663,0.000001,0.000012,0.000010,0.000159,0.000418,0.394627,0.002071,0.000007,0.957648
24181,0.504213,0.001882,0.000006,0.087540,0.004176,0.000025,0.015292,0.003656,0.016812,0.000597,0.878793,0.008567,0.010542,0.045125
24184,0.000492,0.003698,0.001255,0.682195,0.074103,0.000775,0.000134,0.000294,0.043057,0.000085,0.002233,0.000215,0.013227,0.000047


# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [ ]:
results_filepaths_mean_pred = glob.glob('./logged_performance_per_model/*/*agg_mean*.csv')
results_filepaths_majority_pred = glob.glob('./logged_performance_per_model/*/*agg_majority*.csv')

In [ ]:
def conantenate_results(filepath_list):
    dfs_list = []
    for results_filepath in filepath_list:
        model_name = results_filepath.split('/')[-2]
        results_df_i = pd.read_csv(results_filepath)
        results_df_i['model_name'] = model_name
        dfs_list.append(results_df_i)

    results_df_ = pd.concat(dfs_list).set_index(['language', 'model_name', 'unit_of_analysis']).sort_index()
    results_df_.rename(columns={'f1-mico_mean': 'f1-micro_mean', 'f1-mico_std': 'f1-micro_std'}, inplace=True)

    return results_df_

In [ ]:
results_mean_pred_df = conantenate_results(results_filepaths_mean_pred)
results_majority_vote_pred_df = conantenate_results(results_filepaths_majority_pred)

### Generate the tables to report

In [ ]:
def display_performance_table(df, metric, index_cols=['model_name'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'{metric}_mean'].map(lambda x: f'{x:.2f}') + \
    ' $\pm$ ' + report_table[f'{metric}_std'].map(lambda x: f'{x:.2f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

### Generate tables for all languages

In [ ]:
metrics_to_report = ['f1-micro', 'recall-micro', 'precision-micro', 'roc-auc', 'accuracy']

In [ ]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [ ]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby(level=0):
        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=results_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [ ]:
display_metrics_and_write_to_file(df=results_mean_pred_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables_mean_prediction')

In [ ]:
display_metrics_and_write_to_file(df=results_majority_vote_pred_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables_majority_voting')